In [ ]:
import numpy as np 
import pandas as pd
from datetime import timedelta
import datetime as dt
import matplotlib.pyplot as plt
import pickle as cPickle
from os.path import exists
from pathlib import Path
import csv
import datetime
import pytz

from pathlib import Path 
import time

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold,cross_val_score
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/TNC_Collab_Results/training_data_all_20220203.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103356 entries, 0 to 103355
Columns: 300 entries, Unnamed: 0 to wb2sum
dtypes: float64(290), int64(5), object(5)
memory usage: 236.6+ MB


In [ ]:
data['NewEco3'].unique()

array(['11.1.3', '6.2.12', '10.1.5', '11.1.1', '10.2.1', '7.1.8',
       '6.2.11', '6.2.7', '6.2.8', '10.2.2'], dtype=object)

In [ ]:
records_with_missing_data = len(data[data.isna().any(axis=1)])
print('Dropping {} records with missing data'.format(records_with_missing_data))
data = data.dropna(axis=0,how='any').copy()
#data.isnull().sum().sum()
data['flow_cfs'] = data['flow_cfs'] / data['DRAIN_SQKM']

Dropping 3 records with missing data


In [ ]:
df.head()

,month,flow_cfs,comid,DRAIN_SQKM,SLOPE_PCT_30M,ELEV_MEAN_M_BASIN_30M,ELEV_MIN_M_BASIN_30M,ELEV_MAX_M_BASIN_30M,ElvRng,TOT_ECOL3_1,...,wb10,wb11,wb12,p6sum,p3sum,p2sum,wb6sum,wb3sum,wb2sum,NewEco3_label
0,1,0.010533,22593497,44.1009,45.96,2066.05,171.75,3264.80,3093.05,0.0,...,36.469271,13.544983,6.353392,173.076519,164.956173,145.791419,48.690715,22.244474,19.426220,4
1,1,0.001901,22593537,244.3446,28.19,1186.88,189.20,2453.58,2264.38,0.0,...,18.295394,20.328231,8.441266,83.834380,83.283639,71.869830,6.863135,4.810008,4.094633,4
2,1,0.095711,22592131,23.1543,55.34,1336.75,177.49,2568.37,2390.88,0.0,...,26.513515,21.377368,9.551242,95.562592,93.623910,80.350174,7.734838,4.840720,3.977235,4
3,1,0.085372,22684930,59.4738,54.26,1934.41,1232.32,2860.50,1628.18,0.0,...,55.791413,18.298641,4.255288,197.917833,196.351228,190.395853,56.538718,46.032462,44.989842,4
4,1,0.035991,22681934,127.8981,46.37,1677.59,996.51,2509.58,1513.07,0.0,...,36.697448,18.553784,3.334407,155.306407,154.299421,149.492604,30.426053,24.340323,23.668884,4
5,1,0.072178,20267023,42.4575,44.33,2557.56,1162.12,4126.58,2964.46,0.0,...,1.141672,2.034422,3.674352,201.673616,190.623810,180.559667,163.108466,72.920863,51.780318,6
6,1,0.090435,10744456,166.2210,27.26,2635.82,2009.73,3524.19,1514.46,0.0,...,0.599262,0.517794,1.035589,152.254654,135.837821,123.396514,112.854104,22.142895,15.049105,0
7,1,0.155092,8924041,169.9299,27.69,2457.90,1750.21,3314.72,1564.51,0.0,...,3.801182,1.238397,2.476793,201.814896,174.104761,159.210717,197.704051,41.792674,29.085070,6
8,1,0.099866,8933736,140.1876,20.42,2013.69,1700.08,2768.52,1068.44,0.0,...,7.542906,0.932284,1.864568,178.311970,152.456648,144.663358,131.915637,25.356233,17.289350,6
9,1,0.006103,20350533,276.4206,29.62,474.50,41.42,1373.55,1332.13,0.0,...,4.510752,7.917516,4.656072,120.453788,120.300793,113.920618,6.363320,6.052888,5.712251,3


In [ ]:
# Fixed random state for hyper-parameter search function
comid_unique = data.comid.unique() 
train_comid_f, valid_test_comid = train_test_split(comid_unique, test_size=0.2, random_state=322087861)
test_comid_f, valid_comid_f = train_test_split(valid_test_comid, test_size=0.5, random_state=1840907562)

train_comid_f,valid_comid_f,test_comid_f

(array([22057147, 17078183, 22514414, 17684378, 22524649, 17688327,
        17683402,  2705477, 17694079, 17611425,  2705471, 14993059,
         8056721,  2782657, 20343211, 17113015, 17160150,  2809681,
         8205487, 17694891, 14922321, 17574345,  8294919,  2665525,
        17607917, 17682474,  2803945, 17166342, 17661675,  8056781,
        17609017,  8009213, 17586470, 22048943,  2827982, 22591931,
         2806807,  1671593, 22050535,  2800225,  8209965,  7964867,
         2804901,  8315607,  8287256, 22563182,  1893770,  8315847,
         8944527, 17100623, 20343093, 22595619, 14982092,  8056723,
        21610027,  8246426,  8941693, 20331242,  8944453, 22684930,
        17608281,   347487, 14992381, 17116209,  8037541,  2847739,
         7968461, 20351605, 22555310,  8933522,   342587, 17595405,
        14991725,  8005975,  8287590,  8064109, 20196320,  2809859,
        17100137,  2789355,  2704735, 22590267,  8037183,  8019804,
         8285120,  8261865,   344905,  1670171, 

In [ ]:
def train_test_random_split():

  n1 = np.random.randint(0,2**32)

  comid_unique = data.comid.unique() 
  train_comid, test_comid = train_test_split(comid_unique, test_size=0.2, random_state=n1)

  return train_comid,test_comid

In [ ]:
def x_train(month, comid):
  train = df[df["comid"].isin(comid)]
  train = train[train['month']==month]
  col = ['comid'] 
  train = train.drop(columns = col)
  train.reset_index(drop=True,inplace=True)
  x_train = train.drop(columns = ['flow_cfs']).copy()
  return x_train

def y_train(month, comid):
  train = df[df["comid"].isin(comid)]
  train = train[train['month']==month]
  col = ['comid'] 
  train = train.drop(columns = col)
  train.reset_index(drop=True,inplace=True)
  y_train = train.flow_cfs.values 
  return y_train

def x_valid(month, comid):
  valid = df[df["comid"].isin(comid)]
  valid = valid[valid['month']==month]
  col = ['comid'] 
  valid = valid.drop(columns = col)
  valid.reset_index(drop=True,inplace=True)
  x_valid = valid.drop(columns = ['flow_cfs']).copy()
  return x_valid

def y_valid(month, comid):
  valid = df[df["comid"].isin(comid)]
  valid = valid[valid['month']==month]
  col = ['comid'] 
  valid = valid.drop(columns = col)
  valid.reset_index(drop=True,inplace=True)
  y_valid = valid.flow_cfs.values
  return y_valid

def x_test(month, comid):  
  test = df[df["comid"].isin(comid)]
  test = test[test['month']==month]
  col = ['comid'] 
  test = test.drop(columns = col)
  test.reset_index(drop=True,inplace=True)
  x_test = test.drop(columns = ['flow_cfs']).copy()
  return x_test

def y_test(month, comid):
  test = df[df["comid"].isin(comid)]
  test = test[test['month']==month]
  col = ['comid'] 
  test = test.drop(columns = col)
  test.reset_index(drop=True,inplace=True)  
  y_test = test.flow_cfs.values
  return y_test


In [ ]:
def hyperparameter_csv_append(month, model_run_date_time, r2, leaf, features, depth, estimator):
  if exists('/content/drive/MyDrive/TNC_Collab_Results/Results from pure RF model/hyperparameter_'+str(month)+'.csv') == False:

    with open('/content/drive/MyDrive/TNC_Collab_Results/Results from pure RF model/hyperparameter_'+str(month)+'.csv', 'w') as f:
      writer = csv.writer(f)
      header = ["model_run_date_time", "R2", "leaf", "features", "depth", "estimator"]
      writer.writerow(header)
      f.close()

  with open('/content/drive/MyDrive/TNC_Collab_Results/Results from pure RF model/hyperparameter_'+str(month)+'.csv', 'a') as f:
    row = [str(model_run_date_time), str(r2), str(leaf), str(features), str(depth), str(estimator)]
    new_row = ",".join(row)
    f.write(new_row)
    f.write("\n")

def variable_importance_csv_append(month, model_run_date_time, final_r2, variables, importance_scores):

  if exists('/content/drive/MyDrive/TNC_Collab_Results/Results from pure RF model/variable_importance_'+str(month)+'.csv') == False:

    with open('/content/drive/MyDrive/TNC_Collab_Results/Results from pure RF model/variable_importance_'+str(month)+'.csv', 'w') as f:
      writer = csv.writer(f)
      header = ["model_run_date_time", "final_test_r2", "variables", "rf_feature_importance"]
      writer.writerow(header)

  with open('/content/drive/MyDrive/TNC_Collab_Results/Results from pure RF model/variable_importance_'+str(month)+'.csv', 'a') as f:
      count = 0 
      for each_variable in variables:
        row = [str(model_run_date_time), str(final_r2), str(each_variable), str(importance_scores[count])]
        new_row = ",".join(row)
        f.write(new_row)
        f.write("\n")
        count = count + 1

def get_random_values_hyperparameters():
    min_samples_leaf = np.random.randint(low=50, high=100, size=1)[0]
    max_features = np.random.uniform(0.2,0.5) 
    max_depth = np.random.randint(low=6, high=20, size=1)[0]
    return min_samples_leaf, max_features, max_depth

def hyper_parameter_search(n_iter, month): 
 
    for i in range(n_iter):   
        h1, h2, h3 = get_random_values_hyperparameters()
        # n_estimators = The number of trees in the forest. 
        estimators = [250, 500, 800] # we increased this from the range of 100 to 250,500,800

        for count in estimators:
            rf = RandomForestRegressor(n_jobs = -1, n_estimators=count, min_samples_leaf=h1,max_features=h2, max_depth=h3, verbose=1) 
            print("min_samples_leaf %d, max_features %.1f, max_depth %d, estimator %d" % (h1, h2, h3,count))
            rf.fit(x_train(month, train_comid_f), y_train(month, train_comid_f))
            predictions = rf.predict(x_valid(month, valid_comid_f))

            # Mean absolute error (MAE)
            mae = mean_absolute_error(y_valid(month, valid_comid_f), predictions)
            # Mean squared error (MSE)
            mse = mean_squared_error(y_valid(month, valid_comid_f), predictions)
            # R-squared scores 
            r2 = r2_score(y_valid(month, valid_comid_f)+1, predictions+1)

            # Print metrics
            print('Mean Absolute Error:', round(mae, 2))
            print('Mean Squared Error:', round(mse, 2))
            print('R-squared scores:', round(r2, 2))            

            model_run_date_time = datetime.datetime.now(pytz.timezone('US/Pacific'))

            hyperparameter_csv_append(month, model_run_date_time, r2, h1, h2, h3, count)  


def subset_of_variables(month, number_of_runs):

  dfp = pd.read_csv('/content/drive/MyDrive/TNC_Collab_Results/Results from pure RF model/hyperparameter_'+str(month)+'.csv')
  dfp = dfp.sort_values(by="R2", ascending=False)
  
  best_leaf = dfp['leaf'].iloc[0]
  best_features = dfp['features'].iloc[0]
  best_depth = dfp['depth'].iloc[0]
  best_estimator = dfp['estimator'].iloc[0]

  for i in range(number_of_runs):

      train_comid, test_comid = train_test_random_split()
      rf = RandomForestRegressor(n_jobs = -1, min_samples_leaf=best_leaf,max_features=best_features, max_depth=best_depth, n_estimators=best_estimator,verbose=1) # [Kirk- I think changing the verbose option to 1 is better- less stuff to scroll through]
      rf.fit(x_train(month, train_comid), y_train(month, train_comid))

      importances = rf.feature_importances_
      forest_importances = pd.Series(importances, index=x_train(month, train_comid).columns)
      forest_importances.sort_values(ascending=False)
      predictions = rf.predict(x_test(month, test_comid))
      r2_test = r2_score(y_test(month, test_comid)+1, predictions+1)
      final_r2 = r2_test 
      print('R-squared scores:', final_r2)

      sorted_idx = rf.feature_importances_.argsort()[-15:]

      variables = x_train(month, train_comid).columns[sorted_idx]
      importance_scores = rf.feature_importances_[sorted_idx]
      model_run_date_time = datetime.datetime.now(pytz.timezone('US/Pacific'))
      variable_importance_csv_append(month, model_run_date_time, final_r2, variables, importance_scores)

def train_final_model(month):

  df_reduced = pd.read_csv('/content/drive/MyDrive/TNC_Collab_Results/Results from pure RF model/variable_importance_'+str(month)+'.csv')
  df_reduced = df_reduced.sort_values(by="final_test_r2", ascending=False)

  df_reduced = df_reduced[['variables','rf_feature_importance']]
  vi_grouped_by_mean = df_reduced.groupby(['variables']).mean().reset_index()
  asc_vi_df = vi_grouped_by_mean.sort_values('rf_feature_importance', ascending=False)
  reduced_list = asc_vi_df.variables.unique().tolist()
  print("Number of variables for final model: {}".format(len(reduced_list)))
  print(reduced_list)

  dfp = pd.read_csv('/content/drive/MyDrive/TNC_Collab_Results/Results from pure RF model/hyperparameter_'+str(month)+'.csv')
  dfp = dfp.sort_values(by="R2", ascending=False)
  
  best_leaf = dfp['leaf'].iloc[0]
  best_features = dfp['features'].iloc[0]
  best_depth = dfp['depth'].iloc[0]
  best_estimator = dfp['estimator'].iloc[0]

  reduced_columns_list = ['month']
  reduced_columns_list = reduced_columns_list + reduced_list

  train_comid, test_comid = train_test_random_split()

  # Train Reduced model
  rf = RandomForestRegressor(n_jobs = -1, min_samples_leaf=best_leaf,max_features=best_features, max_depth=best_depth, n_estimators=best_estimator,verbose=1) 
  rf.fit(x_train(month, train_comid).loc[:, reduced_columns_list], y_train(month, train_comid))
  predictions = rf.predict(x_test(month, test_comid).loc[:, reduced_columns_list])
  r2 = r2_score(y_test(month, test_comid)+1, predictions+1)
  model_run_date_time = datetime.datetime.now(pytz.timezone('US/Pacific'))
  print("R2 for reduced model=",r2)

  rf_pickle_file = 'mymodel_reduced_'+str(month)+'_r2_'+str(round(r2,4))+'_'+str(model_run_date_time)+'.pkl'
  with open('/content/drive/My Drive/TNC_Collab_Results/Retrain_RF_reduced_features/'+rf_pickle_file, 'wb') as f:
    cPickle.dump(rf, f)

  # Train Full model
  rf2 = RandomForestRegressor(n_jobs = -1, min_samples_leaf=best_leaf,max_features=best_features, max_depth=best_depth, n_estimators=best_estimator,verbose=1) 
  rf2.fit(x_train(month, train_comid), y_train(month, train_comid))
  predictions2 = rf2.predict(x_test(month, test_comid))
  r22 = r2_score(y_test(month, test_comid)+1, predictions2+1)
  model_run_date_time = datetime.datetime.now(pytz.timezone('US/Pacific'))
  print("R2 for full model=",r22)

  rf_pickle_file = 'mymodel_'+str(month)+'_r2_'+str(round(r22,4))+'_'+str(model_run_date_time)+'.pkl'
  with open('/content/drive/My Drive/TNC_Collab_Results/Results from pure RF model/'+rf_pickle_file, 'wb') as f:
    cPickle.dump(rf2, f)



In [ ]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [11]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Selecting variable subset for month {}".format(month))
  subset_of_variables(month, number_of_runs)
  print("Training final model for month {}".format(month))
  train_final_model(month)

In [ ]:
n_iter = 10
number_of_runs = 40
month_list = [12]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Selecting variable subset for month {}".format(month))
  subset_of_variables(month, number_of_runs)
  print("Training final model for month {}".format(month))
  train_final_model(month)



Running hyperparameter search for month 12
min_samples_leaf 85, max_features 0.2, max_depth 17, estimator 100


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    4.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished


Mean Absolute Error: 0.23
Mean Squared Error: 0.21
R-squared scores: 0.79
min_samples_leaf 61, max_features 0.3, max_depth 11, estimator 100


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    5.5s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished


Mean Absolute Error: 0.2
Mean Squared Error: 0.16
R-squared scores: 0.84
min_samples_leaf 65, max_features 0.4, max_depth 7, estimator 100


...

R-squared scores: 0.7899853815299867
Training final model for month 12
Number of variables for final model: 33
['wb0', 'p0', 'WD_OCT_BASIN', 'p2sum', 'WD_JAN_BASIN', 'HLR19', 'p3sum', 'AnnMaxPrecip', 'PPTAVG_BASIN', 'WD_DEC_BASIN', 'CAT_AnnMaxPrecip', 'SILTAVE', 'WD_MAR_BASIN', 'WDMAX_BASIN', 'WD_FEB_BASIN', 'SANDAVE', 'SGEO21', 'p6sum', 'sedimentary', 'PET_CAT', 'p1', 'granitic', 'WD_NOV_BASIN', 'ANN_CONS_WETDAYS', 'wb1', 'CLAYAVE', 'CaO_pct', 'pmpe', 'RH_CAT', 'WD_BASIN', 'PPTAVG_CAT', 'UCS', 'BFI_AVE']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished


R2 for reduced model= 0.7917760809477823


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   10.4s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished


R2 for full model= 0.7982157850454653


In [ ]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [1]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Selecting variable subset for month {}".format(month))
  subset_of_variables(month, number_of_runs)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 1
min_samples_leaf 72, max_features 0.3, max_depth 9, estimator 100


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    8.3s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished


Mean Absolute Error: 0.29
Mean Squared Error: 0.31
R-squared scores: 0.8
min_samples_leaf 60, max_features 0.4, max_depth 10, estimator 100


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    8.2s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished


Mean Absolute Error: 0.26
Mean Squared Error: 0.28
R-squared scores: 0.81
min_samples_leaf 93, max_features 0.4, max_depth 8, estimator 100


...

R-squared scores: 0.8492699361663109
Training final model for month 1
Number of variables for final model: 38
['wb0', 'p0', 'wb1', 'wb2sum', 'p3sum', 'SILTAVE', 'p6sum', 'SANDAVE', 'WD_OCT_BASIN', 'SGEO21', 'WD_JAN_BASIN', 'wb3sum', 'HLR19', 'WD_DEC_BASIN', 'CaO_pct', 'WD_FEB_BASIN', 'BFI_AVE', 'ANN_CONS_WETDAYS', 'p2sum', 'CLAYAVE', 'PPTAVG_BASIN', 'AnnMaxPrecip', 'ELEV_MEAN_M_BASIN_30M', 'WD_MAR_BASIN', 'SiO_pct', 'granitic', 'T_MIN_BASIN', 'CAT_AnnMaxPrecip', 'WDMAX_BASIN', 'TOT_ECOL3_1', 'sedimentary', 'SNOW_PCT_PRECIP', 'PPTAVG_CAT', 'RH_CAT', 'RH_BASIN', 'HGD', 'ET', 'NO10AVE']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    1.9s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished


R2 for reduced model= 0.8108319045753614


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    7.9s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished


R2 for full model= 0.8110117198476317


In [ ]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [2]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Selecting variable subset for month {}".format(month))
  subset_of_variables(month, number_of_runs)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 2
min_samples_leaf 92, max_features 0.3, max_depth 13, estimator 250


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   12.9s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.1s finished


Mean Absolute Error: 0.3
Mean Squared Error: 0.3
R-squared scores: 0.8
min_samples_leaf 92, max_features 0.3, max_depth 13, estimator 500


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:   22.8s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   25.6s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 500 out of 500 | elapsed:    0.2s finished


Mean Absolute Error: 0.3
Mean Squared Error: 0.3
R-squared scores: 0.81
min_samples_leaf 92, max_features 0.3, max_depth 13, estimator 800


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    4.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.1s finished


R2 for reduced model= 0.7902542487636146


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   13.9s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   17.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s


R2 for full model= 0.7855147641850719


[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.1s finished


In [ ]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [3]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Selecting variable subset for month {}".format(month))
  subset_of_variables(month, number_of_runs)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 3
min_samples_leaf 79, max_features 0.3, max_depth 10, estimator 250


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   14.4s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.1s finished


Mean Absolute Error: 0.26
Mean Squared Error: 0.18
R-squared scores: 0.76
min_samples_leaf 79, max_features 0.3, max_depth 10, estimator 500


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:   21.2s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   23.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 500 out of 500 | elapsed:    0.1s finished


Mean Absolute Error: 0.26
Mean Squared Error: 0.18
R-squared scores: 0.76
min_samples_leaf 79, max_features 0.3, max_depth 10, estimator 800


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    7.2s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 500 out of 500 | elapsed:    0.2s finished


R2 for reduced model= 0.7541273014033605


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   11.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:   26.9s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   30.2s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s


R2 for full model= 0.7556597852532878


[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 500 out of 500 | elapsed:    0.2s finished


In [ ]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [4]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Selecting variable subset for month {}".format(month))
  subset_of_variables(month, number_of_runs)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 4
min_samples_leaf 81, max_features 0.3, max_depth 6, estimator 250


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   12.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.1s finished


Mean Absolute Error: 0.31
Mean Squared Error: 0.21
R-squared scores: 0.68
min_samples_leaf 81, max_features 0.3, max_depth 6, estimator 500


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:   21.6s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   24.3s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 500 out of 500 | elapsed:    0.1s finished


Mean Absolute Error: 0.3
Mean Squared Error: 0.2
R-squared scores: 0.68
min_samples_leaf 81, max_features 0.3, max_depth 6, estimator 800


...

R-squared scores: 0.6387785782570901
Training final model for month 4
Number of variables for final model: 40
['wb0', 'p6sum', 'pmpe', 'p0', 'RECHARGE', 'PPTAVG_BASIN', 'WD_MAR_BASIN', 'AnnMinPrecip', 'PPTAVG_CAT', 'T_MAX_BASIN', 'WD_APR_BASIN', 'WDMIN_BASIN', 'WD_BASIN', 'WDMAX_BASIN', 'WD_MAY_BASIN', 'LST32F_BASIN', 'AnnMaxPrecip', 'CAT_AnnMaxPrecip', 'SGEO8', 'SNOW_PCT_PRECIP', 'WD_JUL_BASIN', 'WD_JUN_BASIN', 'FST32F_BASIN', 'T_MIN_BASIN', 'NO200AVE', 'OMAVE', 'CAT_AnnMinPrecip', 't2', 'p3sum', 'WD_SEP_BASIN', 'T_AVG_BASIN', 't0', 't1', 'ELEV_MEAN_M_BASIN_30M', 'wb5', 'ET', 'wb1', 'p1', 'T_AVG_CAT', 'TOT_ECOL3_5']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    4.6s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.1s finished


R2 for reduced model= 0.7253232792617094


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   14.3s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   18.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.1s finished


R2 for full model= 0.7270853455564998


In [ ]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [5]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Selecting variable subset for month {}".format(month))
  subset_of_variables(month, number_of_runs)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 5
min_samples_leaf 70, max_features 0.3, max_depth 7, estimator 250


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    9.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.1s finished


Mean Absolute Error: 0.35
Mean Squared Error: 0.33
R-squared scores: 0.82
min_samples_leaf 70, max_features 0.3, max_depth 7, estimator 500


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:   17.6s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   19.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 500 out of 500 | elapsed:    0.1s finished


Mean Absolute Error: 0.34
Mean Squared Error: 0.32
R-squared scores: 0.83
min_samples_leaf 70, max_features 0.3, max_depth 7, estimator 800


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    3.2s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.1s finished


R2 for reduced model= 0.792960938733783


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   13.6s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s


R2 for full model= 0.7944631470156719


[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.1s finished


In [ ]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [6]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Selecting variable subset for month {}".format(month))
  subset_of_variables(month, number_of_runs)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 6
min_samples_leaf 87, max_features 0.3, max_depth 13, estimator 250


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   13.5s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.1s finished


Mean Absolute Error: 0.34
Mean Squared Error: 0.44
R-squared scores: 0.79
min_samples_leaf 87, max_features 0.3, max_depth 13, estimator 500


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:   26.1s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   28.9s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 500 out of 500 | elapsed:    0.1s finished


Mean Absolute Error: 0.33
Mean Squared Error: 0.44
R-squared scores: 0.79
min_samples_leaf 87, max_features 0.3, max_depth 13, estimator 800


...

R-squared scores: 0.7903120145485494
Training final model for month 6
Number of variables for final model: 26
['wb0', 't2', 'p6sum', 't0', 'T_MAX_BASIN', 't1', 'p3sum', 'SGEO8', 't3', 'p2sum', 'PET', 'SNOW_PCT_PRECIP', 'ELEV_MEAN_M_BASIN_30M', 'p3', 'CLAYAVE', 'SANDAVE', 'T_MIN_BASIN', 'T_AVG_BASIN', 'NO200AVE', 'p2', 'RECHARGE', 'SILTAVE', 'p5', 'wb1', 'FST32F_BASIN', 'p4']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    4.4s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.1s finished


R2 for reduced model= 0.6835944800770889


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   14.4s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   18.4s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.1s finished


R2 for full model= 0.6704802205772513


In [ ]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [7]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Selecting variable subset for month {}".format(month))
  subset_of_variables(month, number_of_runs)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 7
min_samples_leaf 95, max_features 0.2, max_depth 11, estimator 250


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    8.9s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.1s finished


Mean Absolute Error: 0.24
Mean Squared Error: 0.26
R-squared scores: 0.55
min_samples_leaf 95, max_features 0.2, max_depth 11, estimator 500


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:   15.9s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   17.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 500 out of 500 | elapsed:    0.1s finished


Mean Absolute Error: 0.24
Mean Squared Error: 0.26
R-squared scores: 0.54
min_samples_leaf 95, max_features 0.2, max_depth 11, estimator 800


...

R-squared scores: 0.4915485303715871
Training final model for month 7
Number of variables for final model: 32
['wb0', 't1', 'p6sum', 't3', 'wb1', 't2', 'p3sum', 'T_MAX_BASIN', 't0', 'SGEO8', 'p4', 'ELEV_MAX_M_BASIN_30M', 'ELEV_MEAN_M_BASIN_30M', 't4', 'T_MIN_BASIN', 'SNOW_PCT_PRECIP', 'LST32F_BASIN', 'p6', 'RECHARGE', 'DRAIN_SQKM', 'T_AVG_BASIN', 'HGD', 'BFI_AVE', 't8', 'WDMIN_BASIN', 'p3', 'FST32F_BASIN', 't9', 'HLR13', 'wb2sum', 'PET', 'ElvRng']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    5.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.1s finished


R2 for reduced model= 0.6742507557271684


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   17.6s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.1s finished


R2 for full model= 0.685142247426312


In [ ]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [8]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Selecting variable subset for month {}".format(month))
  subset_of_variables(month, number_of_runs)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 8
min_samples_leaf 84, max_features 0.4, max_depth 14, estimator 250


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   14.9s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   18.9s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.1s finished


Mean Absolute Error: 0.13
Mean Squared Error: 0.05
R-squared scores: 0.3
min_samples_leaf 84, max_features 0.4, max_depth 14, estimator 500


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   14.7s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:   34.7s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   39.4s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 500 out of 500 | elapsed:    0.3s finished


Mean Absolute Error: 0.13
Mean Squared Error: 0.05
R-squared scores: 0.31
min_samples_leaf 84, max_features 0.4, max_depth 14, estimator 800


...

R-squared scores: 0.2811243225062804
Training final model for month 8
Number of variables for final model: 53
['ELEV_MAX_M_BASIN_30M', 'wb0', 'LST32F_BASIN', 'wb1', 'SNOW_PCT_PRECIP', 'SGEO8', 'FST32F_BASIN', 'WDMIN_BASIN', 'T_MIN_BASIN', 'p6sum', 'TOT_ECOL3_6', 'RECHARGE', 'wb2sum', 't2', 'WD_SEP_BASIN', 'HGC', 'T_MAX_BASIN', 'HGD', 'HLR17', 'RH_CAT', 'ElvRng', 'SGEO21', 'CONTACT', 'T_AVG_BASIN', 'HLR16', 'CAT_AnnMaxPrecip', 'HLR18', 'DRAIN_SQKM', 'UCS', 'AnnMaxPrecip', 'PPTAVG_CAT', 'sedimentary', 'ELEV_MIN_M_BASIN_30M', 'AnnMinPrecip', 'RFACT', 'PET', 'WD_NOV_BASIN', 'WD_MAY_BASIN', 'BFI_AVE', 'wb3sum', 't4', 'DEPTH_WATTAB', 't1', 'HLR10', 'WD_JUN_BASIN', 'WD_AUG_BASIN', 'CaO_pct', 'HLR9', 'p7', 'wb2', 'HLR19', 'HLR20', 'HLR13']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    3.2s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.1s finished


R2 for reduced model= 0.19776311227782128


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   10.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.1s finished


R2 for full model= 0.19291843576711698


In [ ]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [9]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Selecting variable subset for month {}".format(month))
  subset_of_variables(month, number_of_runs)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 9
min_samples_leaf 76, max_features 0.2, max_depth 13, estimator 250


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   11.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.1s finished


Mean Absolute Error: 0.09
Mean Squared Error: 0.02
R-squared scores: 0.21
min_samples_leaf 76, max_features 0.2, max_depth 13, estimator 500


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:   19.6s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   22.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 500 out of 500 | elapsed:    0.2s finished


Mean Absolute Error: 0.09
Mean Squared Error: 0.02
R-squared scores: 0.21
min_samples_leaf 76, max_features 0.2, max_depth 13, estimator 800


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:    7.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 800 out of 800 | elapsed:    0.2s finished


R2 for reduced model= 0.057565066255937936


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:   18.3s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:   32.6s
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:   32.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s


R2 for full model= 0.02406691205985345


[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 800 out of 800 | elapsed:    0.2s finished


In [ ]:
n_iter = 10
number_of_runs = 40
#month_list = [11, 12, 1, 2, 3, 4]
month_list = [10]

for month in month_list:
  print("Running hyperparameter search for month {}".format(month))
  hyper_parameter_search(n_iter, month)
  print("Selecting variable subset for month {}".format(month))
  subset_of_variables(month, number_of_runs)
  print("Training final model for month {}".format(month))
  train_final_model(month)

Running hyperparameter search for month 10
min_samples_leaf 96, max_features 0.3, max_depth 17, estimator 250


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   15.4s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.1s finished


Mean Absolute Error: 0.07
Mean Squared Error: 0.01
R-squared scores: 0.43
min_samples_leaf 96, max_features 0.3, max_depth 17, estimator 500


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:   29.4s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   32.6s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 500 out of 500 | elapsed:    0.1s finished


Mean Absolute Error: 0.07
Mean Squared Error: 0.01
R-squared scores: 0.43
min_samples_leaf 96, max_features 0.3, max_depth 17, estimator 800


...

R-squared scores: 0.31253488599297596
Training final model for month 10
Number of variables for final model: 46
['p0', 'wb0', 'WD_JUN_BASIN', 'WD_SEP_BASIN', 'WD_MAY_BASIN', 'AnnMinPrecip', 'WD_BASIN', 'HLR19', 'RECHARGE', 'CAT_AnnMinPrecip', 'WD_APR_BASIN', 'HGC', 'WD_OCT_BASIN', 'HLR16', 'volcanic', 'HLR9', 'HGA', 'TOPWET', 'WDMIN_BASIN', 'pmpe', 'WD_NOV_BASIN', 'HLR15', 'CaO_pct', 'DEPTH_WATTAB', 'HGD', 'SLOPE_PCT_30M', 'Lperm', 'RH_CAT', 'wb2sum', 'RFACT', 'BFI_AVE', 'ELEV_MAX_M_BASIN_30M', 'PPTAVG_CAT', 'T_MIN_BASIN', 'NO4AVE', 'NO10AVE', 'HGB', 'PPTAVG_BASIN', 'wb1', 'SILTAVE', 'wb5', 'wb3sum', 'wb4', 'HLR18', 'AnnMaxPrecip', 'ELEV_MEAN_M_BASIN_30M']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    2.2s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.1s finished


R2 for reduced model= 0.3362634928799031


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    8.5s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 250 out of 250 | elapsed:    0.1s finished


R2 for full model= 0.3410565378538082
